In [1]:
import numpy as np
import torch 
import pyro
import re
import pandas as pd

import importlib
import time
import pickle

from granch_utils import init_model_tensor, main_sim_tensor, lesioned_sim, compute_prob_tensor, init_params_tensor, num_stab_help, proxy_sim
#importlib.reload(granch_utils)
importlib.reload(num_stab_help)
importlib.reload(init_model_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)

<module 'granch_utils.lesioned_sim' from '/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/lesioned_sim.py'>

# Main TEST

In [38]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(init_model_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)


device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_INFO = {
    "jitter_n": 1, 
    "total_batch_n": 1, 
    "jitter_mode": "sampling"
}

GRID_INFO = {
    "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 5, 
    "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 5, 
    "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 5, 
    "grid_epsilon_start": 0.00000000001, "grid_epsilon_end": 1, "grid_epsilon_step": 5, 
    "hypothetical_obs_grid_n": 10
}


BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

PRIOR_INFO = {
    "mu_prior": 1,  
    "V_prior": 1, 
    "alpha_prior": 3, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":1, "sd_epsilon": 4, 
    "hypothetical_obs_grid_n": 5, 
    "world_EIGs": 0.00001, "max_observation": 500
}

tensor_stimuli = num_stab_help.sample_spore_experiment(pair_each_stim = 1, n_feature=3)


tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= 5, 
                linking_hypothesis = "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()
pd.set_option('display.max_rows', None)

if params.linking_hypothesis == "EIG": 

   res = main_sim_tensor.granch_main_simulation(params, tensor_model, tensor_stimuli[0])
   #lesioned_res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])
#  print(lesioned_res.behavior)

   print(res.behavior)
else: 
    print(params.linking_hypothesis)
    res = proxy_sim.granch_proxy_sim(params, tensor_model, tensor_stimuli[0])
    print(res.behavior)

#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])



res.behavior.to_csv("behavioral_fit/s.csv")

surprisal
25 -3.0796784259965587e-06
26 -3.0528592305208725e-06
27 -3.0519011519552516e-06
28 -3.0525168399929045e-06
29 -3.05293964005827e-06
30 -3.052873281689166e-06
31 -3.052496979931421e-06
32 -3.052799983517164e-06
33 -3.053175867604511e-06
34 -3.053390411101257e-06
35 -3.052893166673697e-06
36 -3.0535481655088264e-06
37 -3.0531899188607484e-06
38 -3.0531507757322237e-06
39 -3.052992131559431e-06
40 -3.0529071083865553e-06
41 -3.0535603098931545e-06
42 -3.0531049436194126e-06
43 -3.053032533662578e-06
44 -3.0536597777203e-06
45 -3.0527230172651577e-06
46 -3.0538890689114035e-06
47 -3.0531357646775833e-06
48 -3.052986210799827e-06
49 -3.053142410210834e-06
50 -3.0529565575248065e-06
51 -3.052777226144556e-06
52 -3.052827414581319e-06
53 -3.053406972423052e-06
54 -3.0521052791216447e-06
55 -3.052941615518924e-06
56 -3.054143658137228e-06
57 -3.0526582639877792e-06
58 -3.0533909211920447e-06
59 -3.0529750655503464e-06
60 -3.0532858080661836e-06
61 -3.0523330247966734e-06
62 -3.05241

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:370: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.20207899691286055' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.current_stimulus_idx].tolist()
/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:370: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.09714013332238819' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.current_stimulus_idx].tolist()
/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:370: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.0276100208232346' has

167 -3.0531155127918627e-06
168 -3.0531028842748617e-06
169 -3.0537327974536657e-06
170 -3.053180890128087e-06
171 -3.0533629188128393e-06
172 -3.0531408998264643e-06
173 -3.0529119137265505e-06
174 -3.0524600894372926e-06
175 -3.0524835758993176e-06
176 -3.052664820748529e-06
177 -3.0532288815495015e-06
178 -3.0527351951092985e-06
179 -3.052849966951398e-06
180 -3.052724324094708e-06
181 -3.0529103891564054e-06
182 -3.0522906147589064e-06
183 -3.0527599432461375e-06
184 -3.052648859814991e-06
185 -3.051920458536877e-06
186 -3.0532656080447045e-06
187 -3.054256685254038e-06
188 -3.0530275125701757e-06
189 -3.052530394805989e-06
190 -3.053683427796218e-06
191 -3.052554314341222e-06
192 -3.0524644926358676e-06
193 -3.0537578947835714e-06
194 -3.053320971955741e-06
195 -3.0529103744168704e-06
196 -3.0525752125843236e-06
197 -3.0527090188098036e-06
198 -3.0531920108169335e-06
199 -3.0530372692280444e-06
200 -3.0528275474409076e-06
201 -3.05329883413932e-06
202 -3.0528800615016684e-06
203 -

In [17]:
import matplotlib.pyplot as plt


ImportError: dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kiwisolver/_cext.cpython-310-darwin.so, 0x0002): tried: '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kiwisolver/_cext.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kiwisolver/_cext.cpython-310-darwin.so' (no such file), '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/kiwisolver/_cext.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))

# Specific Grid Test

In [548]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(init_model_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)
from torch.distributions import Normal, uniform


# test stimuli
test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBB")
test_stimuli.add_toy_example(0.3, 0.7)

# test grid  
grid_mu_distribution = uniform.Uniform(.2, .8)
grid_sigma_distribution = uniform.Uniform(.2, .8)
grid_y_distribution = uniform.Uniform(.2, .8)
grid_epsilon_distribution = uniform.Uniform(.2, .8)

# grid_mu = grid_mu_distribution.sample([2, ])
# grid_sigma = grid_sigma_distribution.sample([2, ])
# grid_y = grid_y_distribution.sample([2, ])
# grid_epsilon = grid_epsilon_distribution.sample([2, ])

grid_mu = torch.linspace(start = .2, end  = .8, steps = 5)
grid_sigma = torch.linspace(start = .2, end  = .8, steps = 5)
grid_y = torch.linspace(start = .2, end  = .8, steps = 5)
grid_epsilon = torch.linspace(start = .2, end  = .8, steps = 5)

# test prior
PRIOR_INFO = {
    "mu_prior": 1,  
    "V_prior": 1, 
    "alpha_prior": 1, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
    "hypothetical_obs_grid_n": 2, 
    "world_EIGs": 0.001, "max_observation": 500
}


tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], test_stimuli)

params = init_params_tensor.granch_params(
                grid_mu =  grid_mu,
                grid_sigma = grid_sigma,
                grid_y = grid_y, 
                grid_epsilon = grid_epsilon,
                hypothetical_obs_grid_n = 5, 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan, 
                linking_hypothesis = "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

if params.linking_hypothesis == "EIG": 
    res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
else: 
    res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)

res.behavior


/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30011168122291565' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29988840222358704' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2999730110168457' has dtype incompatible with int64, please explicitly cast to a compatible d

tensor(-12.4852, dtype=torch.float64)
tensor(-13.2946, dtype=torch.float64)
tensor(-13.3100, dtype=torch.float64)
tensor(-13.3129, dtype=torch.float64)
tensor(-13.3140, dtype=torch.float64)
tensor(-13.3179, dtype=torch.float64)
tensor(-13.3183, dtype=torch.float64)
tensor(-13.3217, dtype=torch.float64)
tensor(-13.3198, dtype=torch.float64)
tensor(-13.3197, dtype=torch.float64)
tensor(-13.3204, dtype=torch.float64)
tensor(-13.3200, dtype=torch.float64)
tensor(-13.3205, dtype=torch.float64)
tensor(-13.3215, dtype=torch.float64)
tensor(-13.3208, dtype=torch.float64)
tensor(-13.3238, dtype=torch.float64)
tensor(-13.3208, dtype=torch.float64)
tensor(-13.3202, dtype=torch.float64)
tensor(-13.3217, dtype=torch.float64)
tensor(-13.3218, dtype=torch.float64)
tensor(-13.3205, dtype=torch.float64)
tensor(-13.3230, dtype=torch.float64)
tensor(-13.3198, dtype=torch.float64)
tensor(-13.3220, dtype=torch.float64)
tensor(-13.3225, dtype=torch.float64)
tensor(-13.3235, dtype=torch.float64)
tensor(-13.3

,stimulus_id,EIG,Look_away,surprisal,kl
0,0,NaN,False,-12.485166,1.434026e-01
1,0,NaN,False,-13.294644,8.591513e-04
2,0,NaN,False,-13.309959,1.427475e-04
3,0,NaN,False,-13.312906,3.384517e-05
4,0,NaN,False,-13.314026,7.788265e-06
5,0,NaN,False,-13.317922,1.050467e-06
6,0,NaN,False,-13.318296,2.218626e-07
7,0,NaN,False,-13.321696,1.064257e-06
8,0,NaN,False,-13.319807,2.046861e-06
9,0,NaN,False,-13.319726,2.779853e-06


# Visualizing random grid KL (500)

In [544]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(proxy_sim)


def run_random_grid_and_visualize_kl():


    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    BATCH_INFO = {
        "jitter_n": 1, 
        "total_batch_n": 1, 
        "jitter_mode": "sampling"
    }

    GRID_INFO = {
       "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 5, 
       "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 5, 
       "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 5, 
       "grid_epsilon_start": 0.000001, "grid_epsilon_end": 1, "grid_epsilon_step": 5, 
       "hypothetical_obs_grid_n": 10
    }


#    GRID_INFO = {
#        "grid_mu_start": .2, "grid_mu_end": .8, "grid_mu_step": 5, 
#        "grid_sigma_start": .2, "grid_sigma_end": .8, "grid_sigma_step": 5, 
#        "grid_y_start":.2, "grid_y_end": 8, "grid_y_step": 5, 
#        "grid_epsilon_start": .2, "grid_epsilon_end": .8, "grid_epsilon_step": 5, 
#        "hypothetical_obs_grid_n": 10
#    }



    BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

    PRIOR_INFO = {
        "mu_prior": 1,  
        "V_prior": 1, 
        "alpha_prior": 3, 
        "beta_prior": 1, 
        "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
        "hypothetical_obs_grid_n": 2, 
        "world_EIGs": 0.01, "max_observation": 500
    }


    test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
    test_stimuli.add_toy_example(0.3, 0.7)
    



    tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], test_stimuli)

    params = init_params_tensor.granch_params(
                    grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                    grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                    grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                    grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                    hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                    mu_prior = PRIOR_INFO["mu_prior"],
                    V_prior = PRIOR_INFO["V_prior"], 
                    alpha_prior = PRIOR_INFO["alpha_prior"], 
                    beta_prior = PRIOR_INFO["beta_prior"],
                    epsilon  = PRIOR_INFO["epsilon"], 
                    mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                    sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                    world_EIGs = PRIOR_INFO["world_EIGs"],
                    max_observation = PRIOR_INFO["max_observation"], 
                    forced_exposure_max= np.nan, 
                    linking_hypothesis = "surprisal")
            
                # add the various different cached bits
    params.add_meshed_grid()
    params.add_lp_mu_sigma()
    params.add_y_given_mu_sigma()
    params.add_lp_epsilon()
    params.add_priors()

    if params.linking_hypothesis == "EIG": 
        res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
    else: 
        res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)

    return res


#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])



# try it out 5 eims
res_list = []
#kl_df_list = []
for i in range(2000): 
    print(i)
    res= (run_random_grid_and_visualize_kl()).behavior
    res["id"] = i
    res_list.append(res)
    #kl_df_list.append(kl_df)

main_res = pd.concat(res_list)
#kl_df_res = pd.concat(kl_df_list)

main_res.to_csv("diagnostics/sim_res.csv")
#main_res.dropna()
#print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)



0
tensor(-4.3281e-08, dtype=torch.float64)
tensor(-4.3281e-08, dtype=torch.float64)
tensor(-4.3281e-08, dtype=torch.float64)
tensor(-4.3281e-08, dtype=torch.float64)
tensor(-4.3281e-08, dtype=torch.float64)
tensor(-4.3281e-08, dtype=torch.float64)
1
tensor(-2.0384, dtype=torch.float64)
tensor(-2.4123, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4120, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4123, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
tensor(-2.4119, dtype=torch.float64)
tensor(-2.4120, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4120, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
tensor(-2.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29983359575271606' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30007535219192505' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3001207709312439' has dtype incompatible with int64, please explicitly cast to a compatible d

tensor(-2.4123, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
tensor(-2.4124, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4120, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
tensor(-2.4120, dtype=torch.float64)
tensor(-2.4120, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4123, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4120, dtype=torch.float64)
tensor(-2.4120, dtype=torch.float64)
tensor(-2.4120, dtype=torch.float64)
tensor(-2.4120, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
tensor(-2.4120, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
tensor(-2.4121, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
tensor(-2.4122, dtype=torch.float64)
t

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30001121759414673' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30006733536720276' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29989752173423767' has dtype incompatible with int64, please explicitly cast to a compatible 

tensor(-8.1743, dtype=torch.float64)
tensor(-8.1819, dtype=torch.float64)
tensor(-8.1764, dtype=torch.float64)
tensor(-8.1757, dtype=torch.float64)
tensor(-8.1745, dtype=torch.float64)
tensor(-8.1786, dtype=torch.float64)
tensor(-8.1840, dtype=torch.float64)
tensor(-8.1849, dtype=torch.float64)
tensor(-8.1729, dtype=torch.float64)
tensor(-8.1730, dtype=torch.float64)
tensor(-8.1743, dtype=torch.float64)
tensor(-8.1754, dtype=torch.float64)
tensor(-8.1859, dtype=torch.float64)
tensor(-8.1829, dtype=torch.float64)
tensor(-8.1746, dtype=torch.float64)
tensor(-8.1708, dtype=torch.float64)
tensor(-8.1719, dtype=torch.float64)
tensor(-8.1770, dtype=torch.float64)
tensor(-8.1740, dtype=torch.float64)
tensor(-8.1790, dtype=torch.float64)
tensor(-8.1699, dtype=torch.float64)
tensor(-8.1716, dtype=torch.float64)
tensor(-8.1686, dtype=torch.float64)
tensor(-8.1730, dtype=torch.float64)
tensor(-8.1767, dtype=torch.float64)
tensor(-8.1738, dtype=torch.float64)
tensor(-8.1680, dtype=torch.float64)
t

...

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30005332827568054' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.30007392168045044' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3000729978084564' has dtype incompatible with int64, please explicitly cast to a compatible d

tensor(-0.6492, dtype=torch.float64)
tensor(-0.6490, dtype=torch.float64)
tensor(-0.6492, dtype=torch.float64)
tensor(-0.6492, dtype=torch.float64)
tensor(-0.6492, dtype=torch.float64)
tensor(-0.6488, dtype=torch.float64)
tensor(-0.6493, dtype=torch.float64)
tensor(-0.6491, dtype=torch.float64)
tensor(-0.6493, dtype=torch.float64)
tensor(-0.6489, dtype=torch.float64)
tensor(-0.6494, dtype=torch.float64)
tensor(-0.6494, dtype=torch.float64)
tensor(-0.6490, dtype=torch.float64)
tensor(-0.6497, dtype=torch.float64)
tensor(-0.6488, dtype=torch.float64)
tensor(-0.6491, dtype=torch.float64)
tensor(-0.6493, dtype=torch.float64)
tensor(-0.6491, dtype=torch.float64)
tensor(-0.6489, dtype=torch.float64)
tensor(-0.6492, dtype=torch.float64)
tensor(-0.6494, dtype=torch.float64)
tensor(-0.6491, dtype=torch.float64)
tensor(-0.6492, dtype=torch.float64)
tensor(-0.6494, dtype=torch.float64)
tensor(-0.6492, dtype=torch.float64)
tensor(-0.6494, dtype=torch.float64)
tensor(-0.6495, dtype=torch.float64)
t

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.2999093532562256' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.3000396192073822' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/Users/caoanjie/Desktop/projects/looking_time_models/03_pyGRANCH_multi/granch_utils/init_model_tensor.py:302: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29998478293418884' has dtype incompatible with int64, please explicitly cast to a compatible dt

tensor(-1.1465, dtype=torch.float64)
tensor(-1.1464, dtype=torch.float64)
tensor(-1.1464, dtype=torch.float64)
tensor(-1.1464, dtype=torch.float64)
tensor(-1.1465, dtype=torch.float64)
tensor(-1.1464, dtype=torch.float64)
tensor(-1.1465, dtype=torch.float64)
tensor(-1.1464, dtype=torch.float64)
tensor(-1.1466, dtype=torch.float64)
tensor(-1.1466, dtype=torch.float64)
tensor(-1.1465, dtype=torch.float64)
tensor(-1.1466, dtype=torch.float64)
tensor(-1.1465, dtype=torch.float64)
tensor(-1.1466, dtype=torch.float64)
tensor(-1.1465, dtype=torch.float64)
tensor(-1.1465, dtype=torch.float64)
tensor(-1.1464, dtype=torch.float64)
tensor(-1.1464, dtype=torch.float64)
tensor(-1.1465, dtype=torch.float64)
tensor(-1.1466, dtype=torch.float64)
tensor(-1.1464, dtype=torch.float64)
tensor(-1.1464, dtype=torch.float64)
tensor(-1.1465, dtype=torch.float64)
tensor(-1.1465, dtype=torch.float64)
tensor(-1.1465, dtype=torch.float64)
tensor(-1.1464, dtype=torch.float64)
tensor(-1.1465, dtype=torch.float64)
t

KeyboardInterrupt: 

In [ ]:
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

df = main_res

# Get unique categories
id = df['id'].unique()
# Create a subplot grid
fig = sp.make_subplots(rows=len(id), cols=1, subplot_titles=["1", "2", "3", "4", "5"])

# Populate each subplot
for i, idx in enumerate(id):
    subset = df[df['id'] == idx]
    print(subset)
    single_fig = px.line(df, x='t', y=['kl1', 'kl2'], labels={'kl1': 'kl1', 'kl2': 'kl2'}, markers=True)
    single_fig.update_layout(
    xaxis_title='t',
    yaxis_title='Value',
    legend_title='Legend',
    )


    for trace in single_fig.data:
        fig.add_trace(trace, row=i + 1, col=1)

# Update subplot layout (optional)
fig.update_layout(
    title='Faceted Line Plots',
    showlegend=True,
    height=600,
    width=400,
)

# Show the plot
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px



df = res.iloc[:50]

# Create a line plot using Plotly Express
fig = px.line(df, x='t', y=['kl1', 'kl2'], labels={'kl1': 'kl1', 'kl2': 'kl2'}, markers=True)

# Customize the layout (optional)
fig.update_layout(
    title='Line Plot with Plotly',
    xaxis_title='t',
    yaxis_title='Value',
    legend_title='Legend',
)

# Show the plot
fig.show()


# Toy Example Test 

In [179]:
importlib.reload(init_model_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(proxy_sim)

test_stimuli = init_model_tensor.granch_stimuli(3, "BBBBBD")
test_stimuli.add_toy_example(0.3, 0.7)

tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = 3,
                #world_EIGs = PRIOR_INFO["world_EIGs"],
                #max_observation = PRIOR_INFO["max_observation"], 
                max_observation = 500,
                forced_exposure_max= np.nan, 
                linking_hypothesis= "surprisal")
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

pd.set_option('display.max_rows', None)

#res = main_sim_tensor.granch_main_simulation(params, tensor_model, test_stimuli)
res = proxy_sim.granch_proxy_sim(params, tensor_model, test_stimuli)
#res.behavior

res.behavior

NameError: name 'tensor_stimuli' is not defined

In [ ]:
res.behavior

In [303]:
import torch.nn.functional as F

new_post = torch.tensor([[[[ 4.1338e-01, 9.9999e-321],
          [ 3.6698e-02, 9.9999e-321]],

         [[ 4.8830e-01, 9.9999e-321],
          [ 6.1620e-02, 9.9999e-321]]],


        [[[ 4.1338e-01, 9.9999e-321],
          [ 3.6698e-02, 9.9999e-321]],

         [[ 4.8830e-01, 9.9999e-321],
          [ 6.1619e-02, 9.9999e-321]]],


        [[[ 4.1338e-01, 9.9999e-321],
          [ 3.6698e-02, 9.9999e-321]],

         [[ 4.8830e-01, 9.9999e-321],
          [ 6.1620e-02, 9.9999e-321]]]], dtype=torch.float64)

paded_prev_post = torch.tensor([[[[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]],

         [[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]]],


        [[[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]],

         [[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]]],


        [[[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]],

         [[1.0000e+00, 1.0000e-31],
          [1.3534e-01, 1.0000e-31]]]])

#print(new_post/paded_prev_post)
#print(torch.log(new_post/paded_prev_post))

KL1 = torch.sum(torch.mul(new_post,
                         torch.log(new_post/paded_prev_post)), dim = (1, 2, 3))

KL2 = F.kl_div(torch.log(paded_prev_post),new_post, reduction = "none").sum(dim = (1, 2, 3))

print(KL1)
print(KL2)

print(new_post.sum())
print(paded_prev_post.sum(dim = (1, 2, 3)))
print(new_post < paded_prev_post)

print(torch.log(new_post/paded_prev_post))

tensor([-0.8116, -0.8116, -0.8116], dtype=torch.float64)
tensor([-0.8116, -0.8116, -0.8116], dtype=torch.float64)
tensor(3.0000, dtype=torch.float64)
tensor([2.2707, 2.2707, 2.2707])
tensor([[[[True, True],
          [True, True]],

         [[True, True],
          [True, True]]],


        [[[True, True],
          [True, True]],

         [[True, True],
          [True, True]]],


        [[[True, True],
          [True, True]],

         [[True, True],
          [True, True]]]])
tensor([[[[  -0.8834, -665.4471],
          [  -1.3051, -665.4471]],

         [[  -0.7168, -665.4471],
          [  -0.7868, -665.4471]]],


        [[[  -0.8834, -665.4471],
          [  -1.3051, -665.4471]],

         [[  -0.7168, -665.4471],
          [  -0.7868, -665.4471]]],


        [[[  -0.8834, -665.4471],
          [  -1.3051, -665.4471]],

         [[  -0.7168, -665.4471],
          [  -0.7868, -665.4471]]]], dtype=torch.float64)


# Test KL overflow

In [241]:

A = torch.tensor([9.9999e-321,  2.4348e-01])
B = torch.tensor([1.0000e-31, 3.6788e-01])
        
A_cast_first = torch.tensor([9.9999e-321,  2.4348e-01], dtype=torch.float64)
B_cast_first = torch.tensor([1.0000e-31, 3.6788e-01], dtype=torch.float64)  

A_cast_later = A.to(torch.float64)
B_cast_later = B.to(torch.float64)


mask = A_cast_later < 1e-10
A_cast_later[mask] = 1e-10 
mask = B_cast_later < 1e-10
B_cast_later[mask] = 1e-10


KL1 = torch.sum(torch.mul(A,torch.log(A/B)))
KL2 = torch.sum(torch.mul(A_cast_later,torch.log(A_cast_later/B_cast_later)))
KL3 =  torch.sum(torch.mul(A_cast_first,torch.log(A_cast_first/B_cast_first)))

print(KL1)
print(KL2)
print(KL3)

tensor(nan)
tensor(-0.1005, dtype=torch.float64)
tensor(-0.1005, dtype=torch.float64)


In [168]:
A = torch.tensor([9.9999e-321])
B = A.to(torch.float64)
print(A)
print(B)
print(B.dtype)

tensor([0.])
tensor([0.], dtype=torch.float64)
torch.float64


In [124]:
pred = torch.tensor([[0.2, 0.8]])
target = torch.tensor([[0.1, 0.9]])

kl2_a = F.kl_div(pred, target, reduction='sum', log_target=False)

kl2_b = F.kl_div(target, pred, reduction='sum', log_target=True)

KL1 = torch.sum(torch.mul(new_post, torch.log(new_post/paded_prev_post)), dim = 0)

print(kl2_a)
print(kl2_b)

print(torch.sum(torch.exp(target) * (target - pred)))


tensor(-1.0651)
tensor(-0.1004)
tensor(0.1354)
